In [98]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy  as np
import pandas as pd
from scipy import stats
from statsmodels.graphics.gofplots import qqplot
from sklearn import linear_model, metrics
plt.style.use('ggplot')
%matplotlib inline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold


In [99]:
raw = pd.read_csv("./house-prices-advanced-regression-techniques/cleandata/s2_clean_dummified.csv")

In [100]:
raw.shape

(1458, 145)

In [101]:
pd.set_option('display.max_columns', 150)
raw.head()

,Id,LotFrontage,LotArea,OverallQual,TotalBsmtSF,GrLivArea,TotRmsAbvGrd,GarageCars,WoodDeckSF,OpenPorchSF,ScreenPorch,PoolQC,MiscVal,Age,Re_Age,TotalFullBath,TotalHalfBath,Artery,Feedr,PosA,PosN,RRAe,RRAn,RRNe,RRNn,LogSalePrice,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_None,FireplaceQu_Po,FireplaceQu_TA,LandContour_HLS,LandContour_Low,LandContour_Lvl,LotShape_IR2,LotShape_IR3,LotShape_Reg,PavedDrive_Y,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_None,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_None,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtExposure_None,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,BsmtCond_Gd,BsmtCond_None,BsmtCond_Po,BsmtCond_TA,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,SeasonSold_Spring,SeasonSold_Summer,SeasonSold_Winter,Functional_Typ,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,LandSlope_Mod,LandSlope_Sev,Street_Pave,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,CentralAir_Y,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_None,BsmtQual_TA,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,65.0,8450,7,856,1710,8,2,0,61,0,0,0,5,5,3,1,0,0,0,0,0,0,0,0,12.247694,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0
1,2,80.0,9600,6,1262,1262,6,2,298,0,0,0,0,31,31,2,1,0,1,0,0,0,0,0,0,12.109011,0,0,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0
2,3,68.0,11250,7,920,1786,6,2,0,42,0,0,0,7,6,3,1,0,0,0,0,0,0,0,0,12.317167,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0
3,4,60.0,9550,7,756,1717,7,3,0,35,0,0,0,91,36,2,0,0,0,0,0,0,0,0,0,11.849398,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0
4,5,84.0,14260,8,1145,2198,9,3,192,84,0,0,0,8,8,3,1,0,0,0,0,0,0,0,0,12.429216,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0


In [102]:
sale_price = raw['LogSalePrice']
raw = raw.drop(['LogSalePrice','Id'],axis=1)

In [103]:
#Remove columns from train that are not in test data rather than making them 0 in test
raw = raw.drop(['Exterior1st_ImStucc', 'Exterior1st_Stone', 'HouseStyle_2.5Fin'] ,axis=1)

In [104]:
def find_outliers(df,col):
    mean = np.mean(df[col], axis=0)
    sd = np.std(df[col], axis=0)
    gap=mean-4*sd
    gap2=mean+4*sd
    outliers = df[ (df[col] > gap2) | (df[col]< gap)].index
    return outliers

miscval=list(find_outliers(raw, 'MiscVal'))
lotarea=list(find_outliers(raw, 'LotArea'))
totalbsmtsf=list(find_outliers(raw,'TotalBsmtSF'))
grlivarea=list(find_outliers(raw,'GrLivArea'))
droplist=miscval+lotarea+totalbsmtsf+grlivarea
raw.drop(droplist,axis=0, inplace=True)

In [105]:
raw.shape

(1437, 140)

In [43]:
X = raw.copy()
Y = sale_price.copy()

In [106]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
ridge = Ridge()
lasso = Lasso()
net   = ElasticNet()

In [45]:
#running ridge with alpha 0 (MLR)
ridge.set_params(alpha = 0, normalize = True)
ridge.fit(X, Y)
ridge.score(X, Y)

0.9300159162360475

In [48]:
#Grid search for ridge
alphas_ridge = np.linspace(0,10,50)
tuned_parameters_r = [{'alpha': alphas_ridge}]
n_folds = 5
cv = KFold(n_splits=n_folds, shuffle=True)

tune_ridge = GridSearchCV(ridge, tuned_parameters_r, 
                          cv=cv, refit=True, return_train_score = True, 
                          scoring = 'neg_mean_squared_error')
tune_ridge.fit(X,Y)

In [80]:
print(tune_ridge.best_params_)
print(np.max(tune_ridge.cv_results_['mean_test_score']))
print(np.min(tune_ridge.cv_results_['mean_test_score']))

{'alpha': 0.20408163265306123}


In [81]:
ridge_best = tune_ridge.best_estimator_
ridge_best.fit(X,Y)
ridge_best.score(X,Y)

0.9195114466268476

In [107]:
#fitting lasso with alpha 0 (MLR)
lasso.set_params(alpha = 0, normalize = True, max_iter = 10000)
lasso.fit(X, Y)
lasso.score(X, Y)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:478: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


0.9300345228143976

In [108]:
#Grid search for lasso

alphas_lasso = np.logspace(-7, -2, 50)
tuned_parameters_l = [{'alpha': alphas_lasso}]
n_folds = 5
cv = KFold(n_splits=n_folds, shuffle=True)

tune_lasso = GridSearchCV(lasso, tuned_parameters_l, 
                          cv=cv, refit=True, return_train_score = True, 
                          scoring = 'neg_mean_squared_error')
tune_lasso.fit(X,Y)

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
       error_score='raise-deprecating',
       estimator=Lasso(alpha=0, copy_X=True, fit_intercept=True, max_iter=10000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'alpha': array([1.00000e-07, 1.26486e-07, 1.59986e-07, 2.02359e-07, 2.55955e-07,
       3.23746e-07, 4.09492e-07, 5.17947e-07, 6.55129e-07, 8.28643e-07,
       1.04811e-06, 1.32571e-06, 1.67683e-06, 2.12095e-06, 2.68270e-06,
       3.39322e-06, 4.29193e-06, 5.42868e-06, 6.86649e-06, 8.6...2.44205e-03, 3.08884e-03,
       3.90694e-03, 4.94171e-03, 6.25055e-03, 7.90604e-03, 1.00000e-02])}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

In [109]:
print(tune_lasso.best_params_)
#print(np.max(-tune_lasso.cv_results_['mean_test_score']))
print(np.min(-tune_lasso.cv_results_['mean_test_score']))

{'alpha': 1.757510624854793e-05}
0.014700755129557494


In [110]:
lasso_best = tune_lasso.best_estimator_
lasso_best.fit(X,Y)
lasso_best.score(X,Y)

0.9284965416097749

In [93]:
elastic = linear_model.ElasticNet(alpha = 0, l1_ratio=1, normalize=True)
elastic.fit(X,Y)
elastic.score(X,Y)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:478: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


0.9300345228143841

In [96]:
#Elastic net grid search

#N_alpha = 100
#N_rho   = 10
#alphaRange = np.logspace(-10, 2, N_alpha)
#rhoRange   = np.linspace(0.1,1, N_rho) # we avoid very small rho by starting at 0.1
#scores     = np.zeros((N_rho, N_alpha))
#for alphaIdx, alpha in enumerate(alphaRange):
#    for rhoIdx, rho in enumerate(rhoRange):
#            elastic.set_params(alpha=alpha, l1_ratio=rho)
#            elastic.fit(X, Y)
#            scores[rhoIdx, alphaIdx] = elastic.score(X, Y)

In [69]:
raw_test = pd.read_csv(".//house-prices-advanced-regression-techniques/cleandata/s2_clean_dummified_test.csv")

In [70]:
test_IDs = raw_test['Id']
test_IDs = pd.DataFrame(test_IDs)
raw_test.drop(['Id', 'Exterior1st_Other'], axis = 1, inplace = True)

In [111]:
## running lasso to predict housing values 
predict_lasso = lasso_best.predict(raw_test)
predict_lasso = np.exp(predict_lasso)
predict_lasso = pd.DataFrame(predict_lasso)
predict_lasso = pd.concat([test_IDs, predict_lasso], axis = 1)
predict_lasso.columns = ['Id', 'SalePrice']
predict_lasso.to_csv('remove cols in train not in test, remove outliers lasso.csv', index=False)

In [86]:
## running ridge to predict housing values 
predict_ridge = ridge_best.predict(raw_test)
predict_ridge = np.exp(predict_ridge)
predict_ridge = pd.DataFrame(predict_ridge)
predict_ridge = pd.concat([test_IDs, predict_ridge], axis = 1)
predict_ridge.columns = ['Id', 'SalePrice']
predict_ridge.to_csv('remove cols in train not in test ridge.csv', index=False)